# Import packages

In [45]:
import os
import sklearn as skl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Part A: Exploratory Data Analysis
1. Perform Exploratory Data Analysis (EDA) and discuss the data and what you observe
prior to beginning modeling and how the EDA insights impact how to proceed 

### Aisles

In [46]:
aisles = pd.read_csv(r"C:\Users\rmcnei\Documents\CSC84040\instacart-market-basket-analysis\aisles.csv")

print(f"No duplicate aisles: {len(pd.unique(aisles.aisle)) == len(aisles.aisle)}")

print(f"NA values: {aisles.isnull().sum().sum()}")

aisles.head()

No duplicate aisles: True
NA values: 0


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


Aisles appears to be a table of aisles (by name (str)) with unique id numbers assigned to each.

### Departments

In [47]:
departments = pd.read_csv(r"C:\Users\rmcnei\Documents\CSC84040\instacart-market-basket-analysis\departments.csv")

print(f"No duplicate departments: {len(pd.unique(departments.department)) == len(departments.department)}")

print(f"NA values: {departments.isnull().sum().sum()}")

departments.head()

No duplicate departments: True
NA values: 0


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


Departments appears to be a table of departments (by name (str)) with unique id numbers assigned to each.

### Order_products__prior

In [ ]:
order_products__prior = pd.read_csv(r"C:\Users\rmcnei\Documents\CSC84040\instacart-market-basket-analysis\order_products__prior.csv")

In [91]:
print(f"NA values: {order_products__prior.isnull().sum().sum()}")

print(f"Orders in dataset: {order_products__prior.order_id.nunique()}")

print(f"Frequency of reorder: {round(order_products__prior.reordered.mean(), 4) * 100}%")

order_products__prior.describe()

NA values: 0
Orders in dataset: 3214874
Frequency of reorder: 58.97%


,order_id,product_id,add_to_cart_order,reordered
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07
mean,1.710749e+06,2.557634e+04,8.351076e+00,5.896975e-01
std,9.873007e+05,1.409669e+04,7.126671e+00,4.918886e-01
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.559430e+05,1.353000e+04,3.000000e+00,0.000000e+00
50%,1.711048e+06,2.525600e+04,6.000000e+00,1.000000e+00
75%,2.565514e+06,3.793500e+04,1.100000e+01,1.000000e+00
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00


order_products__prior appears to be a table of products previously ordered by order (for ~3.4 million orders), including the order in which each product was added to cart & a binary flag for whether that product was previously ordered by that customer.

### Order_products__train

In [86]:
order_products__train = pd.read_csv(r"C:\Users\rmcnei\Documents\CSC84040\instacart-market-basket-analysis\order_products__train.csv")

In [100]:
print(f"NA values: {order_products__train.isnull().sum().sum()}")

print(f"Orders in dataset: {order_products__train.order_id.nunique()}")

print(f"Frequency of reorder: {round(order_products__train.reordered.mean(), 4) * 100}%")

print(f"Comparative size: {round((order_products__train.__len__() / order_products__prior.__len__()) * 100, 2)}%")

order_products__train.describe()

NA values: 0
Orders in dataset: 131209
Frequency of reorder: 59.86%
Comparative size: 4.27%


,order_id,product_id,add_to_cart_order,reordered
count,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06
mean,1.706298e+06,2.555624e+04,8.758044e+00,5.985944e-01
std,9.897326e+05,1.412127e+04,7.423936e+00,4.901829e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.433700e+05,1.338000e+04,3.000000e+00,0.000000e+00
50%,1.701880e+06,2.529800e+04,7.000000e+00,1.000000e+00
75%,2.568023e+06,3.794000e+04,1.200000e+01,1.000000e+00
max,3.421070e+06,4.968800e+04,8.000000e+01,1.000000e+00


order_products__train appears to be

### Orders

In [51]:
# orders

### Products

In [52]:
# products

### Sample_submission

In [53]:
# sample_submission

2. Pre-processed data for use in the model and justify pre-processing methods. Note this
may be different for each Part and/or algorithm you try. 

# Part B: Association Rules
Use “order_products__prior.csv” to build frequent itemsets and association rules to understand
what products are often bought together. Also, use your findings to understand what drives
market baskets and association rules that are indicative of large lemon purchases. Please provide
commentary and rationale for everything you are doing.

# Part C: Predictive Modeling
Using “order_products__prior.csv” to train a model and “order_products__train.csv” ONLY for
scoring, train a model to predict if large lemons (hint: careful here) will be in an order (for all
orders in “order_products__train.csv”) given the other items in the order and any other
information in the other datasets provided. Please provide commentary and rationale for
everything you are doing.

# Part D: Results Summary
In this section discuss all of your findings including insights from the above Parts A-C. In what
circumstances would Part B vs Part C make sense? What are the advantages of each?